In [ ]:
! pip install aiohttp -q
! pip install mysql.connector -q
! pip install pymysql -q

     |████████████████████████████████| 1.2MB 5.1MB/s 
     |████████████████████████████████| 256kB 47.0MB/s 
     |████████████████████████████████| 317kB 45.8MB/s 
     |████████████████████████████████| 11.9MB 4.8MB/s 
     |████████████████████████████████| 51kB 3.5MB/s 


Домашнее задание №6
==

**НЕ изменяйте и не удаляйте ячейки, написанные с помощью markdown, только ячейки с кодом**


Вам необходимо будет загрузить для проверки:
1. этот ipynb файл в систему
2. данные в базу данных вида *вашпароль_ht*

* Импортируйте библиотеки для работы с mysql
* Подключитесь к удаленному серверу используя для доступа учетные данные с [этой страницы](https://sss.krasilnikov.spb.ru/classsql). Объект для подключения поместите в переменную db_connection

(**1 балл**)

In [ ]:
import aiohttp
import asyncio
import mysql.connector as sql
import pandas as pd
import sqlalchemy
from itertools import starmap
from termcolor import colored

class DB:
    def __init__(self, config):
        self.config = config
        self.dtypes = {int: 'INT', float: 'FLOAT', str: 'VARCHAR(255)'}
        sqlalchemy_config = {**{'drivername': 'mysql+pymysql'}, **config}
        sqlalchemy_config['username'] = sqlalchemy_config.pop('user')
        self.sqlalchemy_url = sqlalchemy.engine.url.URL(**sqlalchemy_config)
        
    def __enter__(self):
        self.connection = sql.connect(**self.config)
        print('connected')
        return self
        
    def execute(self, query):
        cursor = self.connection.cursor(buffered=True)
        try:
            print(colored(query, 'blue'))
            cursor.execute(query)
            return cursor.fetchall() if cursor.rowcount else None
        except Exception as error:
            print(error)
        finally:
            cursor.close()

    def is_table_exist(self, table):
        tables = self.execute('SHOW TABLES')
        return tables and (table,) in tables

    def drop_table_if_exist(self, table):
        if self.is_table_exist(table):
            self.execute(f'DROP TABLE {table}')
            
    def truncate_table_if_exist(self, table):
        if self.is_table_exist(table):
            self.execute(f'TRUNCATE TABLE {table}')

    def column(self, column, dtype):
        return f'  {column} {self.dtypes[dtype]}'

    def create_or_replace_table(self, table, schema, keys):
        self.drop_table_if_exist(table)
        columns = ',\n'.join(starmap(self.column, schema.items()))
        primary_keys = ', '.join(keys)
        self.execute(f'CREATE TABLE {table} (\n{columns},\n  PRIMARY KEY ({primary_keys})\n);')

    def insert_dataframe(self, table, df):
        engine = sqlalchemy.create_engine(self.sqlalchemy_url)
        with engine.connect() as connection:
            df.to_sql(table, connection, index=False, if_exists='append')
            
    def count(self, table):
        return self.execute(f'SELECT count(*) FROM {table}')[0][0]
        
    def __exit__(self, exception_type, exception_value, traceback):
        self.connection.close()
        print('connection closed')
        
config = {
    'host': 'krasilnikov.spb.ru',
    'port': 3360,
    'user': '4049f890b5f59ee17a2554a1e271cc46',
    'password': '4049f890b5f59ee17a2554a1e271cc46',
    'database': '4049f890b5f59ee17a2554a1e271cc46_ht'
}
        
db_connection = DB(config)

* Откройте [страницу данных](https://sss.krasilnikov.spb.ru/classht/2019-7)
* Создайте на удаленном сервере таблицу с названием data
 * столбец назовите аналогично столбцу на странице с данными, замените пробел на нижнее подчеркивание
 * тип данных подберите из числа тех, что были изучены на лекции 7. Для даты используйте varchar

(**3 балла**)

In [ ]:
schema = {
    'region': str, 
    'country': str,
    'item_type': str,
    'sales_channel': str, 
    'order_priority': str,
    'order_date': str,
    'order_id': int, 
    'ship_date': str, 
    'units_sold': int, 
    'unit_price': float,
    'unit_cost': float,
    'total_revenue': float, 
    'total_cost': float, 
    'total_profit': float
}

with db_connection as db:
    db.create_or_replace_table('data', schema, keys=['order_id'])

ProgrammingError: ignored

* Заполните данными таблицу data с указанной страницы.

(**3 балла**)

In [ ]:
%%time
def fetch(pages, url):
    async def fetch_one(session, url):
        print(f'fetching {url}')
        async with session.get(url) as response:
            return await response.text()
    async def fetch_all(pages):
        async with aiohttp.ClientSession() as session:
            return await asyncio.gather(*[fetch_one(session, f'{url}/{page}') for page in pages])
    responses = asyncio.get_event_loop().run_until_complete(fetch_all(pages))
    
    def to_dataframe(html):
        return pd.read_html(html, header=0)[0]
    df = pd.concat(list(map(to_dataframe, responses))).reset_index(drop=True)
    df.columns = [col.lower().replace(' ', '_') for col in df]
    return df

url='https://sss.krasilnikov.spb.ru/classcl/2019-7/4049f890b5f59ee17a2554a1e271cc46'
df = fetch([1], url)

with db_connection as db:
    db.truncate_table_if_exist('data')
    db.insert_dataframe('data', df)
    print(db.count('data'))

fetching https://sss.krasilnikov.spb.ru/classcl/2019-7/4049f890b5f59ee17a2554a1e271cc46/1


ProgrammingError: ignored

* Реализуйте загрузку всех страниц.

(**3 балла**)

In [ ]:
%%time
df = fetch(range(1, 11), url)

with db_connection as db:
    db.truncate_table_if_exist('data')
    db.insert_dataframe('data', df)
    print(db.count('data'))

fetching https://sss.krasilnikov.spb.ru/classcl/2019-7/4049f890b5f59ee17a2554a1e271cc46/8
fetching https://sss.krasilnikov.spb.ru/classcl/2019-7/4049f890b5f59ee17a2554a1e271cc46/4
fetching https://sss.krasilnikov.spb.ru/classcl/2019-7/4049f890b5f59ee17a2554a1e271cc46/9
fetching https://sss.krasilnikov.spb.ru/classcl/2019-7/4049f890b5f59ee17a2554a1e271cc46/3
fetching https://sss.krasilnikov.spb.ru/classcl/2019-7/4049f890b5f59ee17a2554a1e271cc46/6
fetching https://sss.krasilnikov.spb.ru/classcl/2019-7/4049f890b5f59ee17a2554a1e271cc46/10
fetching https://sss.krasilnikov.spb.ru/classcl/2019-7/4049f890b5f59ee17a2554a1e271cc46/2
fetching https://sss.krasilnikov.spb.ru/classcl/2019-7/4049f890b5f59ee17a2554a1e271cc46/1
fetching https://sss.krasilnikov.spb.ru/classcl/2019-7/4049f890b5f59ee17a2554a1e271cc46/7
fetching https://sss.krasilnikov.spb.ru/classcl/2019-7/4049f890b5f59ee17a2554a1e271cc46/5
connected
SHOW TABLES
TRUNCATE TABLE data
SELECT count(*) FROM data
9980
connection closed
CPU time

* Создайте таблицу categories на сервере c столбцами:
 * Item_Type (категория)
 * Region
 * Most_popular_country (самая часто встречающаяся страна в паре категория-регион)
 * Most_popular_Sales_Channel (самая часто встречающийся канал сбыта в паре категория-регион)
 * Max_Order_ID (максимальный номер заказа в паре категория-регион)
 * Sum_Units_Sold (объем продаж в паре категория-регион)
 * Average_Unit_Price (средняя цена по категории в этом регионе)
* заполните таблицу на основе данных из таблицы data используя sql запросы. Каждый регион должен повторяться в каждой категории ровно один раз. Каждая пара категория-регион должна повторяться ровно один раз

(**3 балла**)

In [ ]:
schema = {
    'item_type': str,
    'region': str,
    'most_popular_country': str,
    'most_popular_sales_channel': str,
    'max_order_id': int,
    'sum_units_sold': int,
    'average_unit_price': float
}

query = """
INSERT INTO categories (
  item_type,
  region,
  most_popular_country,
  most_popular_sales_channel,
  max_order_id,
  sum_units_sold,
  average_unit_price
) WITH countries AS (
  SELECT *
  FROM (
         SELECT
           *,
           row_number() OVER w AS top
         FROM (
                SELECT
                  item_type,
                  region,
                  country,
                  count(*) AS total
                FROM data
                GROUP BY 1, 2, 3
              ) AS countries
         WINDOW w AS ( PARTITION BY item_type, region
           ORDER BY total DESC, country )
       ) AS countries
  WHERE top = 1
), sales_channel AS (
  SELECT *
  FROM (
         SELECT
           *,
           row_number() OVER w AS top
         FROM (
                SELECT
                  item_type,
                  region,
                  sales_channel,
                  count(*) AS total
                FROM data
                GROUP BY 1, 2, 3
              ) AS sales_channel
         WINDOW w AS ( PARTITION BY item_type, region
           ORDER BY total DESC, sales_channel )
       ) AS sales_channel
  WHERE top = 1
), agg AS (
  SELECT
    item_type,
    region,
    max(order_id) AS max_order_id,
    sum(units_sold) AS sum_units_sold,
    avg(unit_price) AS average_unit_price
  FROM data
  GROUP BY 1, 2
), cats AS (
  SELECT
    item_type,
    region,
    country AS most_popular_country,
    sales_channel AS most_popular_sales_channel,
    max_order_id,
    sum_units_sold,
    average_unit_price
  FROM agg
  LEFT JOIN countries USING (item_type, region)
  LEFT JOIN sales_channel USING (item_type, region)
) SELECT *
  FROM cats
ON DUPLICATE KEY UPDATE
  most_popular_country = cats.most_popular_country,
  most_popular_sales_channel = cats.most_popular_sales_channel,
  max_order_id = cats.max_order_id,
  sum_units_sold = cats.sum_units_sold,
  average_unit_price = cats.average_unit_price;
"""

with db_connection as db:
    db.create_or_replace_table('categories', schema, keys=['item_type', 'region'])
    db.execute(query)
    print(db.execute('SELECT * FROM categories LIMIT 10'))

connected
SHOW TABLES
DROP TABLE categories
CREATE TABLE categories (
  item_type VARCHAR(255),
  region VARCHAR(255),
  most_popular_country VARCHAR(255),
  most_popular_sales_channel VARCHAR(255),
  max_order_id INT,
  sum_units_sold INT,
  average_unit_price FLOAT,
  PRIMARY KEY (item_type, region)
);

INSERT INTO categories (
  item_type,
  region,
  most_popular_country,
  most_popular_sales_channel,
  max_order_id,
  sum_units_sold,
  average_unit_price
) WITH countries AS (
  SELECT *
  FROM (
         SELECT
           *,
           row_number() OVER w AS top
         FROM (
                SELECT
                  item_type,
                  region,
                  country,
                  count(*) AS total
                FROM data
                GROUP BY 1, 2, 3
              ) AS countries
         WINDOW w AS ( PARTITION BY item_type, region
           ORDER BY total DESC, country )
       ) AS countries
  WHERE top = 1
), sales_channel AS (
  SELECT *
  FROM (
     